# Vitessce configs for paper figures

In [7]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    DataType as dt,
    FileType as ft,
    MultiImageWrapper,
    OmeTiffWrapper
)
import json
from urllib.parse import quote_plus

In [8]:
data_base_url = "https://vitessce-demo-data.storage.googleapis.com/test-data"

In [9]:
vc = VitessceConfig(schema_version="1.0.15", name="Spraggins et al.", description="Comparison of aggregation methods for an imaging mass spectrometry dataset")
ds_simple = vc.add_dataset(name="SIMPLE").add_object(
    MultiImageWrapper(
        image_wrappers=[
            OmeTiffWrapper(img_url=f"{data_base_url}/VAN0006-LK-2-85-IMS_PosMode_multilayer.pyramid.SIMPLE.ome.tiff", name='SIMPLE'),
        ]
 )
)
ds_gaussian = vc.add_dataset(name="GAUSSIAN").add_object(
    MultiImageWrapper(
        image_wrappers=[
            OmeTiffWrapper(img_url=f"{data_base_url}/VAN0006-LK-2-85-IMS_PosMode_multilayer.pyramid.GAUSSIAN.ome.tiff", name='GAUSSIAN'),
        ]
 )
)
ds_area = vc.add_dataset(name="AREA").add_object(
    MultiImageWrapper(
        image_wrappers=[
            OmeTiffWrapper(img_url=f"{data_base_url}/VAN0006-LK-2-85-IMS_PosMode_multilayer.pyramid.AREA.ome.tiff", name='AREA'),
        ]
 )
)
ds_linear = vc.add_dataset(name="LINEAR").add_object(
    MultiImageWrapper(
        image_wrappers=[
            OmeTiffWrapper(img_url=f"{data_base_url}/VAN0006-LK-2-85-IMS_PosMode_multilayer.pyramid.LINEAR.ome.tiff", name='LINEAR'),
        ]
 )
)
sp_simple = vc.add_view(vt.SPATIAL, dataset=ds_simple)
sl_simple = vc.add_view(vt.LAYER_CONTROLLER, dataset=ds_simple)

sp_gaussian = vc.add_view(vt.SPATIAL, dataset=ds_gaussian)
sl_gaussian = vc.add_view(vt.LAYER_CONTROLLER, dataset=ds_gaussian)

sp_area = vc.add_view(vt.SPATIAL, dataset=ds_area)
sl_area = vc.add_view(vt.LAYER_CONTROLLER, dataset=ds_area)

sp_linear = vc.add_view(vt.SPATIAL, dataset=ds_linear)
sl_linear = vc.add_view(vt.LAYER_CONTROLLER, dataset=ds_linear)

vc.link_views([sp_simple, sp_gaussian, sp_area, sp_linear], [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y])

vc.layout(((sp_simple / sl_simple) | (sp_gaussian / sl_gaussian)) | ((sp_area / sl_area) | (sp_linear / sl_linear)));

In [10]:
#vw = vc.widget(height=800)
#vw

VitessceWidget(config={'version': '1.0.15', 'name': 'Spraggins et al.', 'description': 'Comparison of aggregat…

In [12]:
vc.web_app()

'http://vitessce.io/#?theme=light&url=data:,%7B%22version%22%3A+%221.0.15%22%2C+%22name%22%3A+%22Spraggins+et+al.%22%2C+%22description%22%3A+%22Comparison+of+aggregation+methods+for+an+imaging+mass+spectrometry+dataset%22%2C+%22datasets%22%3A+%5B%7B%22uid%22%3A+%22A%22%2C+%22name%22%3A+%22SIMPLE%22%2C+%22files%22%3A+%5B%7B%22fileType%22%3A+%22raster.json%22%2C+%22options%22%3A+%7B%22schemaVersion%22%3A+%220.0.2%22%2C+%22usePhysicalSizeScaling%22%3A+false%2C+%22images%22%3A+%5B%7B%22name%22%3A+%22SIMPLE%22%2C+%22type%22%3A+%22ome-tiff%22%2C+%22url%22%3A+%22https%3A%2F%2Fvitessce-demo-data.storage.googleapis.com%2Ftest-data%2FVAN0006-LK-2-85-IMS_PosMode_multilayer.pyramid.SIMPLE.ome.tiff%22%2C+%22metadata%22%3A+%7B%22isBitmask%22%3A+false%7D%7D%5D%2C+%22renderLayers%22%3A+%5B%22SIMPLE%22%5D%7D%7D%5D%7D%2C+%7B%22uid%22%3A+%22B%22%2C+%22name%22%3A+%22GAUSSIAN%22%2C+%22files%22%3A+%5B%7B%22fileType%22%3A+%22raster.json%22%2C+%22options%22%3A+%7B%22schemaVersion%22%3A+%220.0.2%22%2C+%22usePh